In [2]:
import glob
import json
import os
import cv2
import yaml
import shutil

# Convert tif labels to coco json format

In [3]:
MASK_EXT = 'tif'
ORIGINAL_EXT = 'tif'
MASK_PATH = 'labels'
IMG_PATH = 'images'

In [4]:
def create_annotation_for_contour(contour, annotation_id: int, image_id):
    bbox = cv2.boundingRect(contour)
    area = cv2.contourArea(contour)
    segmentation = contour.flatten().tolist()

    annotation = {
        "iscrowd": 0,
        "id": annotation_id,
        "image_id": image_id,
        "category_id": 1,
        "bbox": bbox,
        "area": area,
        "segmentation": [segmentation],
    }

    return annotation


def contours_from_mask_image(mask_image_open):
    # Find contours in the mask image
    gray = cv2.cvtColor(mask_image_open, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    contours = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[0]
    return contours


def images_annotations_info(path):
    """
    Process the binary masks and generate images and annotations information.

    :param path: Path to the directory containing images and binary masks
    :return: Tuple containing images info, annotations info, and annotation count
    """
    global image_id, annotation_id
    annotations = []
    images = []


    for mask_image in glob.glob(os.path.join(path, MASK_PATH, f'*.{MASK_EXT}')):
        original_file_name = f'{os.path.basename(mask_image).split(".")[0]}.{ORIGINAL_EXT}'
        mask_image_open = cv2.imread(mask_image)

        # Get image dimensions
        height, width, _ = mask_image_open.shape

        # Create or find existing image annotation
        if original_file_name not in map(lambda img: img['file_name'], images):
            image = {
                "id": image_id + 1,
                "width": width,
                "height": height,
                "file_name": original_file_name,
            }
            images.append(image)
            image_id += 1
        else:
            image = [element for element in images if element['file_name'] == original_file_name][0]

        contours = contours_from_mask_image(mask_image_open)

        # Create annotation for each contour
        for contour in contours:
            annotation = create_annotation_for_contour(contour, annotation_id, image['id'])

            # Add annotation if area is greater than zero
            if annotation["area"] > 0:
                annotations.append(annotation)
                annotation_id += 1

    return images, annotations, annotation_id


def process_masks(mask_path, dest_json):
    # Initialize the COCO JSON format with categories
    coco_format = {
        "info": {},
        "licenses": [],
        "images": [],
        "categories": [{"id": 1, "name": 'Vessel', "supercategory": 'Vessel'}],
        "annotations": [],
    }

    # Create images and annotations sections
    coco_format["images"], coco_format["annotations"], annotation_cnt = images_annotations_info(mask_path)

    # Save the COCO JSON to a file
    with open(dest_json, "w") as outfile:
        json.dump(coco_format, outfile, sort_keys=True, indent=4)

    print(f"Created {annotation_cnt} annotations for images in folder: {mask_path}")

In [5]:
!mkdir -p small/train/images
!mkdir -p small/val/images

!mkdir -p small/train/labels
!mkdir -p small/val/labels

In [6]:
# copy images 0500.tif - 0599.tif
!cp /kaggle/input/blood-vessel-segmentation/train/kidney_1_dense/images/05**.tif /kaggle/working/small/train/images
!cp /kaggle/input/blood-vessel-segmentation/train/kidney_3_sparse/images/05**.tif /kaggle/working/small/val/images


In [7]:
# copy labels 0500.tif - 0599.tif
!cp /kaggle/input/blood-vessel-segmentation/train/kidney_1_dense/labels/05**.tif /kaggle/working/small/train/labels
!cp /kaggle/input/blood-vessel-segmentation/train/kidney_3_dense/labels/05**.tif /kaggle/working/small/val/labels


In [8]:
# Sanity check to see if we have 100 files in each dir
!ls -l small/val/labels | grep "^-" | wc -l
!ls -l small/val/images | grep "^-" | wc -l
!ls -l small/train/labels | grep "^-" | wc -l
!ls -l small/train/images | grep "^-" | wc -l


100
100
100
100


In [9]:
global image_id, annotation_id
image_id = 0
annotation_id = 0

In [10]:
train_path = '/kaggle/working/small/train'
val_path = '/kaggle/working/small/val'

train_json_path = '/kaggle/working/small/train/train.json'
val_json_path = '/kaggle/working/small/val/val.json'

In [11]:
process_masks(train_path, train_json_path)
process_masks(val_path, val_json_path)

Created 10959 annotations for images in folder: /kaggle/working/small/train
Created 21024 annotations for images in folder: /kaggle/working/small/val


In [12]:
!rm /kaggle/working/small/train/labels/*.tif
!rm /kaggle/working/small/val/labels/*.tif

In [13]:
!tail -n 100 /kaggle/working/small/val/val.json

        {
            "file_name": "0589.tif",
            "height": 1706,
            "id": 185,
            "width": 1510
        },
        {
            "file_name": "0507.tif",
            "height": 1706,
            "id": 186,
            "width": 1510
        },
        {
            "file_name": "0593.tif",
            "height": 1706,
            "id": 187,
            "width": 1510
        },
        {
            "file_name": "0500.tif",
            "height": 1706,
            "id": 188,
            "width": 1510
        },
        {
            "file_name": "0505.tif",
            "height": 1706,
            "id": 189,
            "width": 1510
        },
        {
            "file_name": "0533.tif",
            "height": 1706,
            "id": 190,
            "width": 1510
        },
        {
            "file_name": "0539.tif",
            "height": 1706,
            "id": 191,
            "width": 1510
        },
        {
            "file_name": "0587.tif",
        

# Convert annotations from coco json to yolo format using ultralytics `convert_coco`

In [14]:
!pip install ultralytics

  Obtaining dependency information for ultralytics from https://files.pythonhosted.org/packages/57/2e/9d58b392f3cb41f75e1f8e274e528c48c46e47caf581e3e34147780c26dc/ultralytics-8.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for hub-sdk>=0.0.2 from https://files.pythonhosted.org/packages/f1/ef/940bd48e9d0fc93109b77f944a86729fbb1e08bbcbb4f558988ccebe853b/hub_sdk-0.0.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 699.2/699.2 kB 5.6 MB/s eta 0:00:00a 0:00:01


In [15]:
from ultralytics.data.converter import convert_coco

In [16]:
convert_coco(labels_dir='/kaggle/working/small/train', 
             use_segments=True,
             cls91to80=False,)

Annotations /kaggle/working/small/train/train.json: 100%|██████████| 100/100 [00:00<00:00, 182.53it/s]

COCO data converted successfully.
Results saved to /kaggle/working/coco_converted


In [17]:
!cp  /kaggle/working/coco_converted/labels/train/*.txt /kaggle/working/small/train/labels/
!rm -r /kaggle/working/coco_converted

In [18]:
convert_coco(labels_dir='/kaggle/working/small/val', 
             use_segments=True,
             cls91to80=False,)

Annotations /kaggle/working/small/val/val.json: 100%|██████████| 100/100 [00:00<00:00, 183.70it/s]

COCO data converted successfully.
Results saved to /kaggle/working/coco_converted


In [19]:
!cp  /kaggle/working/coco_converted/labels/val/*.txt /kaggle/working/small/val/labels/
!rm -r /kaggle/working/coco_converted

# Write yolo dataset YAML

In [20]:
names = {0: 'Vessel'}

# Number of classes
nc = len(names)

# Create a dictionary with the required content
yaml_data = {
    'names': names,
    'nc': nc,
    'test': '',
    'train': train_path,
    'val': val_path
}

# Write the dictionary to a YAML file
with open('small/small_yolo.yaml', 'w') as file:
    yaml.dump(yaml_data, file, default_flow_style=False)

In [21]:
!cat small/small_yolo.yaml

names:
  0: Vessel
nc: 1
test: ''
train: /kaggle/working/small/train
val: /kaggle/working/small/val


In [22]:
!head /kaggle/working/small/train/labels/0582.txt

0 0.265351 0.853415 0.265351 0.854183 0.266447 0.85495 0.267544 0.854183 0.266447 0.853415
0 0.257675 0.84881 0.258772 0.849578 0.259868 0.849578 0.259868 0.84881
0 0.308114 0.838066 0.309211 0.838833 0.310307 0.838833 0.310307 0.838066
0 0.309211 0.827322 0.308114 0.828089 0.308114 0.830391 0.309211 0.830391 0.309211 0.829624 0.311404 0.828089 0.310307 0.827322
0 0.198465 0.822717 0.196272 0.824252 0.196272 0.825787 0.195175 0.826554 0.195175 0.829624 0.197368 0.828089 0.197368 0.826554 0.198465 0.825787
0 0.253289 0.81581 0.253289 0.816577 0.254386 0.816577 0.254386 0.81581
0 0.343202 0.815042 0.343202 0.818112 0.344298 0.818112 0.345395 0.81888 0.346491 0.81888 0.345395 0.818112 0.345395 0.816577
0 0.216009 0.801995 0.213816 0.80353 0.213816 0.805833 0.211623 0.807368 0.212719 0.808135 0.213816 0.807368 0.214912 0.807368 0.216009 0.8066 0.216009 0.805833 0.217105 0.805065 0.217105 0.801995
0 0.195175 0.795088 0.195175 0.795856 0.196272 0.795088
0 0.226974 0.792786 0.225877 0.793553 

# Train yolov8 model

In [23]:
project = 'hacking_human_vasculature_small'
name = 'small_set'

In [24]:
!pip install -U ipywidgets

  Obtaining dependency information for ipywidgets from https://files.pythonhosted.org/packages/4a/0e/57ed498fafbc60419a9332d872e929879ceba2d73cb11d284d7112472b3e/ipywidgets-8.1.1-py3-none-any.whl.metadata
  Obtaining dependency information for widgetsnbextension~=4.0.9 from https://files.pythonhosted.org/packages/29/03/107d96077c4befed191f7ad1a12c7b52a8f9d2778a5836d59f9855c105f6/widgetsnbextension-4.0.9-py3-none-any.whl.metadata
  Obtaining dependency information for jupyterlab-widgets~=3.0.9 from https://files.pythonhosted.org/packages/e8/05/0ebab152288693b5ec7b339aab857362947031143b282853b4c2dd4b5b40/jupyterlab_widgets-3.0.9-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 1.3 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.9/214.9 kB 3.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 20.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: widgetsnbextension
    Found existing installatio

## Enable wandb logging

In [29]:
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_api_key = user_secrets.get_secret("wandb-api-key")
wandb.login(key=wandb_api_key)

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Load segmentation model weights from pretrained and train on the small set

In [30]:
from ultralytics import YOLO

model = YOLO('yolov8n-seg.pt')


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

In [33]:
results = model.train(data='small/small_yolo.yaml',
                      project=project,
                      name=name,
                      epochs=200,
                      patience=0, #I am setting patience=0 to disable early stopping.
                      batch=4,
                      imgsz=1706,
                      device=[0, 1]
                     )

Ultralytics YOLOv8.1.0 🚀 Python-3.10.12 torch-2.0.0 CUDA:0 (Tesla T4, 15102MiB)
                                                     CUDA:1 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=small/small_yolo.yaml, epochs=200, time=None, patience=0, batch=4, imgsz=1706, save=True, save_period=-1, cache=False, device=[0, 1], workers=8, project=hacking_human_vasculature_small, name=small_set3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, s

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


TensorBoard: Start with 'tensorboard --logdir hacking_human_vasculature_small/small_set3', view at http://localhost:6006/


wandb: Currently logged in as: aaalex-lit. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.16.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240110_110810-ix1kp1lg
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run small_set3
wandb: ⭐️ View project at https://wandb.ai/aaalex-lit/hacking_human_vasculature_small
wandb: 🚀 View run at https://wandb.ai/aaalex-lit/hacking_human_vasculature_small/runs/ix1kp1lg


Overriding model.yaml nc=80 with nc=1
Transferred 381/417 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅
WARNING ⚠️ imgsz=[1706] must be multiple of max stride 32, updating to [1728]


...

                   all        100      10064      0.768      0.212      0.317      0.166      0.695      0.192      0.265      0.113

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      6/200      13.2G      1.483      1.643      1.872     0.8737        170       1728: 100%|██████████| 25/25 [00:11<00:00,  2.25it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:20<00:00,  1.22it/s]


                   all        100      10064      0.748      0.274      0.355      0.193      0.673      0.242      0.308      0.136

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      7/200      8.65G      1.393      1.547      1.658     0.8746        182       1728:  52%|█████▏    | 13/25 [00:05<00:05,  2.21it/s]Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/tqdm/std.py", line 1182, in __iter__
    for obj in iterable:
  File "/opt/conda/lib/python3.10/site-packages/ultralytics/data/build.py", line 50, in __iter__
    yield next(self.iterator)
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 634, in __next__
    data = self._next_data()
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1329, in _next_data
    idx, data = self._get_data()
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1285, in _get_data
    success, data = self._try_get_data()
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1133, in _try_get_data
    data = self._data_queue.get(timeout=timeout)
  File "/opt/con

KeyboardInterrupt: 

In [35]:
wandb.finish()